In [35]:
import pandas as pd
import os

In [37]:
# ========== PASO 1: Carga y análisis de archivo ========== #
# Ruta del archivo
file_path = "/Users/danielcapitanlobato/Documents/DOCUMENTACIÓN IMPORTANTE/Trabajo/ELAN ZAAK/Z-Score/zscore_sample/zscore_sample2023.csv"

In [39]:
# Leemos el archivo sin asignar encabezados aún
df_raw = pd.read_csv(file_path, header=None)

In [41]:
# Analizamos los encabezados
df_raw.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Entity Name,Entity ID,Geography,Company Type,Industry Classification,Company Status,Total Revenue\n($000),Total Assets\n($000),EBIT Margin\n(%),Total Revenue\n($000),Total Debt\n($000),Interest Expense\n($000),Total Current Assets\n($000),Total Current Liabilities\n($000),Retained Earnings\n($000)
1,SP_ENTITY_NAME,SP_ENTITY_ID,SP_GEOGRAPHY,SP_COMPANY_TYPE,IQ_INDUSTRY_CLASSIFICATION,SP_COMPANY_STATUS,IQ_TOTAL_REV,IQ_TOTAL_ASSETS,IQ_EBIT_MARGIN,IQ_TOTAL_REV,IQ_TOTAL_DEBT,IQ_INTEREST_EXP,IQ_TOTAL_CA,IQ_TOTAL_CL,IQ_RETAINED_EARNINGS
2,NaN,NaN,NaN,NaN,NaN,NaN,Latest Fiscal Year,FY2023,FY2023,FY2023,FY2023,FY2023,FY2023,FY2023,FY2023
3,NaN,NaN,NaN,NaN,NaN,NaN,Current,Current,Current,Current,Current,Current,Current,Current,Current
4,"""AGBank"" OJSC",4538223,Europe,Private Company,Financials,Liquidating,"11,586",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"""AMIO BANK"" CJSC",4559124,Europe,Private Company,Financials,Operating Subsidiary,"43,853","2,046,827",NaN,"29,306","411,372",NaN,"257,027","1,455,113","(48,077)"
6,"""Bank Dabrabyt"" Joint-stock Company",4265923,Europe,Private Company,Financials,Operating,"38,110",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"""DBK-Leasing"" JSC",4414560,Europe,Private Company,Financials,Operating Subsidiary,"114,053","3,754,743",NaN,"81,929","1,695,628",NaN,"3,041,334","437,402","107,767"
8,"""Muganbank"" Open Joint Stock Company",4552311,Europe,Private Company,Financials,Operating,"14,891",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"""Parabank"" OJSC",4558950,Europe,Private Company,Financials,Out of Business,"6,316",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
# Identificamos que a partir del índice 4 comienzan las observaciones de empresas
# Emplearemos el índice 1 (código técnico) y 2 (año) para armar las columnas; para las que no tienen ni año ni código técnico (como las primeras6 columnas) dejamos el texto del índice 0
header_0 = df_raw.iloc[0] # nombre legible
header_1 = df_raw.iloc[1] # código técnico
header_2 = df_raw.iloc[2] # año fiscal del análisis

In [45]:
# Creamos una función para construir el nombre limpio de la columna
def construir_nombre_col(col):
    cod = header_1[col]
    anio = header_2[col]
    nombre_legible = header_0[col]
    
    # Si hay código y año -> usamos ese formato
    if pd.notna(cod) and pd.notna(anio):
        anio_limpio = str(anio).replace("FY", "")
        return f"{cod}_{anio_limpio}"
    
    # Si solo hay código -> usamos el código
    elif pd.notna(cod):
        return str(cod)
    
    # Si no hay ni código ni año fiscal pero sí nombre legible -> usamos nombre legible (Entity, name, etc.)
    elif pd.notna(nombre_legible):
        return str(nombre_legible).strip()
    
    # Si todo está vacío -> asignamos nombre genérico
    else: # si no hay nada
        return f"col_{col}"

In [47]:
# Aplicamos la función a cada una de las columnas
nuevas_columnas = []

for col in df_raw.columns:
    nombre = construir_nombre_col(col) # aplicamos la función
    nuevas_columnas.append(nombre) # guardamos el resultado

In [53]:
df_limpio = df_raw[4:].copy() # generamos una copia de los puros datos sin encabezado (índice 4 en adelante]
df_limpio.columns = nuevas_columnas # asignamos los nuevos nombres
df_limpio.reset_index(drop=True, inplace=True) # reseteamos índice

In [55]:
df_limpio.head()

,SP_ENTITY_NAME,SP_ENTITY_ID,SP_GEOGRAPHY,SP_COMPANY_TYPE,IQ_INDUSTRY_CLASSIFICATION,SP_COMPANY_STATUS,IQ_TOTAL_REV_Latest Fiscal Year,IQ_TOTAL_ASSETS_2023,IQ_EBIT_MARGIN_2023,IQ_TOTAL_REV_2023,IQ_TOTAL_DEBT_2023,IQ_INTEREST_EXP_2023,IQ_TOTAL_CA_2023,IQ_TOTAL_CL_2023,IQ_RETAINED_EARNINGS_2023
0,"""AGBank"" OJSC",4538223,Europe,Private Company,Financials,Liquidating,"11,586",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"""AMIO BANK"" CJSC",4559124,Europe,Private Company,Financials,Operating Subsidiary,"43,853","2,046,827",NaN,"29,306","411,372",NaN,"257,027","1,455,113","(48,077)"
2,"""Bank Dabrabyt"" Joint-stock Company",4265923,Europe,Private Company,Financials,Operating,"38,110",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""DBK-Leasing"" JSC",4414560,Europe,Private Company,Financials,Operating Subsidiary,"114,053","3,754,743",NaN,"81,929","1,695,628",NaN,"3,041,334","437,402","107,767"
4,"""Muganbank"" Open Joint Stock Company",4552311,Europe,Private Company,Financials,Operating,"14,891",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Ahora que vimos que funciona, aplicamos una función generalizada para el resto de los 4 archivos
# Definimos la ruta base
ruta_base = "/Users/danielcapitanlobato/Documents/DOCUMENTACIÓN IMPORTANTE/Trabajo/ELAN ZAAK/Z-Score/zscore_sample/"

# Procesamos archivos del 2019 al 2023
df_2019 = procesar_archivo(ruta_base + "zscore_sample2019.csv", 2019)
df_2020 = procesar_archivo(ruta_base + "zscore_sample2020.csv", 2020)
df_2021 = procesar_archivo(ruta_base + "zscore_sample2021.csv", 2021)
df_2022 = procesar_archivo(ruta_base + "zscore_sample2022.csv", 2022)
df_2023 = procesar_archivo(ruta_base + "zscore_sample2023.csv", 2023)

# Unimos todo en un solo panel
df_panel = pd.concat([df_2019, df_2020, df_2021, df_2022, df_2023], axis=0)
